In [46]:
from qiskit_nature.second_q.operators import FermionicOp

def hamiltonian_original():
    Vbc = [[-.0665824175665474],
           [.0491028058707432]]
    Eb = -.0632948702054110
    U = 0.29340000
    eps_1 = -0.284183
    eps_2 = -0.2632707
    n_bath = 1
    n_impurity = 2
    
    # Initialize the Hamiltonian
    H = FermionicOp({}, num_spin_orbitals=6)

    # Add the on-site interaction terms
    H += U * FermionicOp({"+_0 -_0 +_1 -_1": 1}, num_spin_orbitals=6)
    H += U * FermionicOp({"+_2 -_2 +_3 -_3": 1}, num_spin_orbitals=6)
    H += eps_1 * (FermionicOp({"+_0 -_0": 1}, num_spin_orbitals=6) +
                   FermionicOp({"+_1 -_1": 1}, num_spin_orbitals=6))
    H += eps_2 * (FermionicOp({"+_2 -_2": 1}, num_spin_orbitals=6) +
                   FermionicOp({"+_3 -_3": 1}, num_spin_orbitals=6))

    # Add the coupling terms
    for f in range(n_impurity):
        for d in range(n_bath):
            factor = Vbc[f][d]
            term_even = f"+_{2*f} -_{2*(2+d)} +_{2*(2+d)} -_{2*f}"
            term_odd = f"+_{2*f + 1} -_{2*(2+d) + 1} +_{2*(2+d) + 1} -_{2*f + 1}"
            H += factor * FermionicOp({term_even: 1}, num_spin_orbitals=6)
            H += factor * FermionicOp({term_odd: 1}, num_spin_orbitals=6)

    # Add the last terms for Eb
    H += Eb * (FermionicOp({"+_4 -_4": 1}, num_spin_orbitals=6) +
                FermionicOp({"+_5 -_5": 1}, num_spin_orbitals=6))

    return H

# Create the Hamiltonian
hamiltonian_fermionic_op = hamiltonian_original()
print(hamiltonian_fermionic_op)


Fermionic Operator
number spin orbitals=6, number terms=12
  0.2934 * ( +_0 -_0 +_1 -_1 )
+ 0.2934 * ( +_2 -_2 +_3 -_3 )
+ -0.284183 * ( +_0 -_0 )
+ -0.284183 * ( +_1 -_1 )
+ -0.2632707 * ( +_2 -_2 )
+ -0.2632707 * ( +_3 -_3 )
+ -0.0665824175665474 * ( +_0 -_4 +_4 -_0 )
+ -0.0665824175665474 * ( +_1 -_5 +_5 -_1 )
+ 0.0491028058707432 * ( +_2 -_4 +_4 -_2 )
+ 0.0491028058707432 * ( +_3 -_5 +_5 -_3 )
+ -0.063294870205411 * ( +_4 -_4 )
+ -0.063294870205411 * ( +_5 -_5 )


In [101]:
Spin_Correlator = FermionicOp({}, num_spin_orbitals=6)
Spin_Correlator += FermionicOp({"+_0 +_2": 0.5}, num_spin_orbitals=6)
Spin_Correlator += FermionicOp({"+_1 +_3": 0.5}, num_spin_orbitals=6)
Spin_Correlator += FermionicOp({"+_1 +_2": -0.5}, num_spin_orbitals=6)
Spin_Correlator += FermionicOp({"+_0 +_3": -0.5}, num_spin_orbitals=6)

In [103]:
from qiskit_nature.second_q.operators import SparseLabelOp
from qiskit_algorithms import EigensolverResult, MinimumEigensolverResult
from qiskit_nature.second_q.problems.eigenstate_result import EigenstateResult

class HamiltonianProblem:
    def __init__(self, hamiltonian: SparseLabelOp, aux_ops: SparseLabelOp):
        self._hamiltonian = hamiltonian
        self.aux_ops = aux_ops
        self.properties = []  # Add any properties you may have here

    @property
    def hamiltonian(self) -> SparseLabelOp:
        """Returns the Hamiltonian wrapped by this problem."""
        return self._hamiltonian

    def second_q_ops(self) -> tuple[SparseLabelOp, dict[str, SparseLabelOp]]:
        """Returns the second quantized operators associated with this problem.

        Returns:
            A tuple, with the first object being the main operator and the second being a dictionary
            of auxiliary operators.
        """
        main_op = self.hamiltonian  # Already a SparseLabelOp

        aux_ops: dict[str, SparseLabelOp] = {}

        aux_ops['spin_correlator'] = self.aux_ops
        
        for prop in self.properties:
            aux_ops.update(prop.second_q_ops())

        return main_op, aux_ops
    
    def interpret(
            self,
            raw_result: EigenstateResult | EigensolverResult | MinimumEigensolverResult,
        ) -> EigenstateResult:
            """Interprets an EigenstateResult in the context of this problem.

            Args:
                raw_result: an eigenstate result object.

            Returns:
                An interpreted `EigenstateResult` in the form of a subclass of it. The actual type
                depends on the problem that implements this method.
            """
            return EigenstateResult.from_result(raw_result)
    

# Define the Hamiltonian terms and coefficients
hamiltonian_terms = hamiltonian_original()
print(hamiltonian_terms)

# Create an instance of HamiltonianProblem
problem = HamiltonianProblem(hamiltonian_terms, Spin_Correlator)

# Access the second quantized operators
main_operator, auxiliary_operators = problem.second_q_ops()

print(main_operator)
print(auxiliary_operators)


Fermionic Operator
number spin orbitals=6, number terms=12
  0.2934 * ( +_0 -_0 +_1 -_1 )
+ 0.2934 * ( +_2 -_2 +_3 -_3 )
+ -0.284183 * ( +_0 -_0 )
+ -0.284183 * ( +_1 -_1 )
+ -0.2632707 * ( +_2 -_2 )
+ -0.2632707 * ( +_3 -_3 )
+ -0.0665824175665474 * ( +_0 -_4 +_4 -_0 )
+ -0.0665824175665474 * ( +_1 -_5 +_5 -_1 )
+ 0.0491028058707432 * ( +_2 -_4 +_4 -_2 )
+ 0.0491028058707432 * ( +_3 -_5 +_5 -_3 )
+ -0.063294870205411 * ( +_4 -_4 )
+ -0.063294870205411 * ( +_5 -_5 )
Fermionic Operator
number spin orbitals=6, number terms=12
  0.2934 * ( +_0 -_0 +_1 -_1 )
+ 0.2934 * ( +_2 -_2 +_3 -_3 )
+ -0.284183 * ( +_0 -_0 )
+ -0.284183 * ( +_1 -_1 )
+ -0.2632707 * ( +_2 -_2 )
+ -0.2632707 * ( +_3 -_3 )
+ -0.0665824175665474 * ( +_0 -_4 +_4 -_0 )
+ -0.0665824175665474 * ( +_1 -_5 +_5 -_1 )
+ 0.0491028058707432 * ( +_2 -_4 +_4 -_2 )
+ 0.0491028058707432 * ( +_3 -_5 +_5 -_3 )
+ -0.063294870205411 * ( +_4 -_4 )
+ -0.063294870205411 * ( +_5 -_5 )
{'spin_correlator': FermionicOp({'+_0 +_2': 0.5, '+_1 +_3'

In [125]:
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

exact_solver = NumPyMinimumEigensolver()
calc = GroundStateEigensolver(JordanWignerMapper(), exact_solver)
result = calc.solve(problem)
print(result)

{   'aux_operators_evaluated': [{'spin_correlator': 0.0}],
    'eigenstates': [   (   <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x31872d2b0>,
                           None)],
    'eigenvalues': array([-0.67733099+0.j]),
    'formatting_precision': 12,
    'groundenergy': -0.6773309877719585,
    'groundstate': (   <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x31872d2b0>,
                       None),
    'raw_result': <qiskit_algorithms.minimum_eigensolvers.numpy_minimum_eigensolver.NumPyMinimumEigensolverResult object at 0x3186da540>}


In [113]:
from typing import Callable, Any, Dict
import numpy as np

# Initialize lists to store the data
evaluation_counts = []
params_list = []
estimated_values = []
metadata_list = []

def optimization_callback(evaluation_count: int, params: np.ndarray, estimated_value: float, metadata: Dict[str, Any]) -> None:
    """
    Callback function to access intermediate data at each optimization step.

    Args:
        evaluation_count (int): The current count of evaluations.
        params (np.ndarray): The current optimizer parameters for the ansatz.
        estimated_value (float): The estimated value at this step.
        metadata (Dict[str, Any]): A dictionary containing metadata about the optimization process.
    """
    # Save the data into arrays/lists
    evaluation_counts.append(evaluation_count)
    params_list.append(params.copy())  # Use copy to avoid references
    estimated_values.append(estimated_value)
    metadata_list.append(metadata.copy())  # Use copy if metadata is mutable

    # Optional: Print the data for confirmation
    print(f"Evaluation Count: {evaluation_count}")
    print(f"Parameters: {params}")
    print(f"Estimated Value: {estimated_value}")
    print(f"Metadata: {metadata}")

# Example usage:
callback: Callable[[int, np.ndarray, float, Dict[str, Any]], None] = optimization_callback

# After optimization, you can convert lists to numpy arrays if needed:
evaluation_counts_array = np.array(evaluation_counts)
params_array = np.array(params_list)  # Note: This will be a 2D array if params have different lengths
estimated_values_array = np.array(estimated_values)
metadata_array = np.array(metadata_list)  # This will also be a 2D array or object array

# Example: Print final stored data
print("Final Evaluation Counts:", evaluation_counts_array)
print("Final Parameters Array:", params_array)
print("Final Estimated Values:", estimated_values_array)
print("Final Metadata Array:", metadata_array)

Final Evaluation Counts: []
Final Parameters Array: []
Final Estimated Values: []
Final Metadata Array: []


In [115]:
from qiskit_algorithms.optimizers import SPSA
from qiskit_algorithms import VQE
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.circuit.library import UCCSD
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_algorithms import EigensolverResult

# Your previously defined Hamiltonian as FermionicOp
#hamiltonian_fermionic_op = FermionicOp({...})  # Replace with your actual Hamiltonian terms

# Create a SecondQuantizedOp
#second_quantized_hamiltonian = SecondQuantizedOp(hamiltonian_fermionic_op)

spsa_optimizer = SPSA(maxiter=500)

noiseless_estimator = AerEstimator(
    run_options={"shots": 10000}
)

mapper = JordanWignerMapper()

num_spin_orbitals=3
num_particles=[2,2]

ansatz = UCCSD(
    num_spin_orbitals,
    num_particles,
    mapper,
    initial_state=HartreeFock(
        num_spin_orbitals,
        num_particles,
        mapper,
    ),
)

callback_dict = {
        "eval_count": 0,
        "params": None,
        "meta": {},
        "value": 0,
    }

vqe_solver = VQE(estimator=noiseless_estimator, ansatz=ansatz, optimizer=spsa_optimizer, callback=optimization_callback)

# Create a mapping from FermionicOp to qubits
mapper = JordanWignerMapper()

# Setup the GroundStateEigensolver
calc = GroundStateEigensolver(mapper, vqe_solver)

# Solve the problem using the SecondQuantizedOp
result = calc.solve(problem)

print(result)


Evaluation Count: 1
Parameters: [-2.1268562430281603, -2.0193781522365297, -1.677995447803837, 5.412663269131554, -5.347425400383837, 0.6671853401830774, -1.0176533361671558, -0.1583628058977123]
Estimated Value: -0.6016609600234929
Metadata: {'shots': 10000, 'variance': 0.02469872402513302, 'simulator_metadata': [{'num_bind_params': 1, 'runtime_parameter_bind': False, 'parallel_state_update': 10, 'parallel_shots': 1, 'sample_measure_time': 0.001697042, 'noise': 'ideal', 'batched_shots_optimization': False, 'remapped_qubits': False, 'active_input_qubits': [0, 1, 2, 3, 4, 5], 'device': 'CPU', 'time_taken': 0.010615667, 'measure_sampling': True, 'num_clbits': 6, 'max_memory_mb': 32768, 'input_qubit_map': [[5, 5], [4, 4], [3, 3], [2, 2], [1, 1], [0, 0]], 'num_qubits': 6, 'method': 'statevector', 'required_memory_mb': 1, 'fusion': {'enabled': True, 'threshold': 14, 'applied': False, 'max_fused_qubits': 5}}]}
Evaluation Count: 2
Parameters: [-2.5268562430281607, -1.6193781522365296, -2.0779

In [118]:
import pandas as pd
import os

# After optimization, you can convert lists to numpy arrays if needed:
evaluation_counts_array = np.array(evaluation_counts)
params_array = np.array(params_list)  # Note: This will be a 2D array if params have different lengths
estimated_values_array = np.array(estimated_values)
metadata_array = np.array(metadata_list)  # This will also be a 2D array or object array

# Save the data to CSV files
pd.DataFrame(evaluation_counts_array, columns=['Evaluation Count']).to_csv('results/evaluation_counts.csv', index=False)
pd.DataFrame(params_array).to_csv('results/parameters.csv', index=False)
pd.DataFrame(estimated_values_array, columns=['Estimated Value']).to_csv('results/estimated_values.csv', index=False)
pd.DataFrame(metadata_array).to_csv('results/metadata.csv', index=False)

In [116]:
print("Local AER simulator result:\n")
print(result.groundenergy)

Local AER simulator result:

-0.6740358459549711
